In [ ]:
import torch
import sys
from pathlib import Path
import os
import matplotlib.pyplot as plt
sys.path.insert(0, str(Path(os.path.abspath('')).parent))
from utils import setup_data_loaders, get_sample, class_mask_to_rgb
from dataset.dataset import TeethDataset

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Plot sample

In [ ]:
image, mask, image_input, mask_input = get_sample("1", validation=False, device=DEVICE, go_parent_folder=True, width=521, height=255)

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.subplots(1, 2)
ax[0].imshow(image, cmap="gray")
plt.axis("off")
ax[1].imshow(mask)
plt.axis("off")
plt.show()

Print the dimensions of an image and a mask:

In [ ]:
# Print the dimensions of an image and a mask:
print(f"Image shape: {image.shape}")
print(f"Mask shape: {mask.shape}")

## Dataset

In [ ]:
DATA_FOLDER = os.path.join("..", "data")
    
train_dataset = TeethDataset(DATA_FOLDER, "training", True, image_height=512, image_width=256)

# print the shape of a random image and its mask
image, mask = train_dataset[1]
print(f"Image shape: {image.shape}")
print(f"Mask shape: {mask.shape}")

In [ ]:
image = image.squeeze(0).unsqueeze(-1)

In [ ]:
rgb_mask = class_mask_to_rgb(mask)

In [ ]:
rgb_mask = rgb_mask * 255.0

In [ ]:
print(f"Mask shape: {rgb_mask.shape}")

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.subplots(1, 2)
ax[0].imshow(image, cmap="gray")
plt.axis("off")
ax[1].imshow(rgb_mask)
plt.axis("off")
plt.show()